In [3]:
# importing libraries needed

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import pyspark.sql.functions as F
from pyspark.sql.functions import when, min, max, col, round, lower, trim, countDistinct, count, sum, month, hour
from pyspark.sql import Window

from pyspark.sql.functions import sum as spark_sum, col, when

ModuleNotFoundError: No module named 'pyspark'

# Starting PySpark session

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3-scala2.13.tgz
!tar xf spark-3.5.6-bin-hadoop3-scala2.13.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.6-bin-hadoop3-scala2.13"

import findspark
findspark.init()

The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.5.6-bin-hadoop3-scala2.13.tgz'


Exception: Unable to find py4j in /content/spark-3.5.6-bin-hadoop3-scala2.13\python, your SPARK_HOME may not be configured correctly

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.config("spark.master", 'local').getOrCreate()

# Introduction to EDA

Objectives for this EDA:

1. Understand and doing basic cleaning on the datasets given
- Find missing values, duplicates or data that have inconsistent formats
- Identify and handle outliers
- Checking for invalid timestamps (e.g. delivery occurs before purchase date)

2. Distribution of data and relationship
- Most commmon product categories, payment methods, purchase hour

3. Customer Behaviour Analysis
- How many repeat buyers exist?
- Average time between orders for repeat buyers
- Preference of repeat buyers (What are they repurchasing)

4. Payment analysis
- Does customers who make paymetns with vouchers behave diffferntly?
- Is there a relationship between high installments (high installment value/high amount of installments made) and low repurchases?

5. Delivery pattern
- How long does delivery take?
- Will long delivery duration cause bad reviews or lower chances of repeat orders?
- Distance between customer and seller

6. Review score
- Are review scores linked to repeat purchases?

7. Product category
- Which product categories are the most popular?
- Group categories which are not popular under 'others'
- Do one-hot encoding on product categories

Final EDA Goal:

To prepare, clean, insightful dataset with features that will be helpful in helping the model to find repeat buyers.

### 1. Understanding and doing basic cleaning on datasets

In [ ]:
# christel's code will be in/under this markdown cell

# Find missing values, duplicates or data that have inconsistent formats
# Identify and handle outliers
# Checking for invalid timestamps (e.g. delivery occurs before purchase date)

### 2. Distribution of data and relationship

In [ ]:
# Most commmon product categories, payment methods, purchase hour

# most common cities 
# most common purchase hour
# most common product categories
# most common payment type
# distribution of no. of installments made/value
# distribution of review score

### 3. Customer Behaviour Analysis

In [ ]:
# How many repeat buyers exist?
# Average time between orders for repeat buyers
# Preference of repeat buyers (What are they repurchasing)

### 4. Payment analysis

In [ ]:
# Does customers who make paymetns with vouchers behave diffferntly?
# Is there a relationship between high installments (high installment value/high amount of installments made) and low repurchases?

### 5. Delivery pattern

In [ ]:
# How long does delivery take?
# Will long delivery duration cause bad reviews or lower chances of repeat orders?
# Distance between customer and seller

### 6. Review score

In [ ]:
# Are review scores linked to repeat purchases?

### 7. Product category

In [ ]:
# Which product categories are the most popular?
# Group categories which are not popular under 'others'
# Do one-hot encoding on product categories